# Front matter

_Important_: On first run, set CACHING to True

In [1]:
CACHING = False
EXPERIMENTAL = False

## Set path

In [2]:
PATH_YIELD_ENGINE = 'src'

def set_paths() -> None:
    """
    set correct working directory and python path when started from within PyCharm
    """
    import sys
    import os
    
    if 'cwd' not in globals():
        # noinspection PyGlobalUndefined
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir)
        os.chdir(cwd)
    
    print(f"working dir is '{os.getcwd()}'")
                             
    if PATH_YIELD_ENGINE not in sys.path:
        sys.path.insert(0, PATH_YIELD_ENGINE)
    
    print(f"added `{sys.path[0]}` to python paths")

set_paths()

working dir is 'C:\Users\martin florent\Documents\projects\yield-engine'
added `src` to python paths


## Imports

In [3]:
import logging
import os
import pickle

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMRegressor
from matplotlib import cm
from matplotlib.pyplot import figure
from matplotlib.ticker import ScalarFormatter
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
import matplotlib.pyplot as plt

import tests
from tests.model import make_simple_transformer
from tests.paths import TEST_DATA_CSV
from yieldengine import Sample
from yieldengine.dendrogram import DendrogramDrawer
from yieldengine.dendrogram.style import FeatMapStyle, LineStyle
from yieldengine.preprocessing.impute import SimpleImputerDF
from yieldengine.preprocessing.selection import BorutaDF
from yieldengine.model.prediction import ModelFitCV
from yieldengine.df.pipeline import PipelineDF
from yieldengine.model.selection import Model, ModelGrid, ModelRanker
from yieldengine.model.validation import CircularCrossValidator
from yieldengine.simulation import UnivariateSimulation
from yieldengine.model.inspection import ModelInspector

C:\Users\martin florent\AppData\Local\Continuum\anaconda3\lib\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [4]:
%matplotlib inline

In [5]:
PATH_TMP = 'tmp'
MI_PKL = os.path.join(PATH_TMP, 'model_inspector.pkl')
MP_PKL = os.path.join(PATH_TMP, 'model_predictor.pkl')

In [6]:
logging.basicConfig(level=logging.DEBUG)
logging.getLogger('matplotlib').setLevel(logging.WARNING)

## Load the data file

In [7]:
inputfile_config = tests.read_test_config(section="inputfile")

batch_df = pd.read_csv(
    filepath_or_buffer=TEST_DATA_CSV,
    delimiter=inputfile_config["delimiter"],
    header=inputfile_config["header"],
    decimal=inputfile_config["decimal"],
)

batch_df = batch_df.drop(columns=["Date", "Batch Id"])

# replace values of +/- infinite with n/a, then drop all n/a columns:
batch_df = batch_df.replace([np.inf, -np.inf], np.nan).dropna(
    axis=1, how="all"
)
batch_df.head()

,Step4 Fermentation Sensor Data Phase2 Pressure Val04 (mbar),Step4-6 RawMat Vendor Compound09 Purity Loss on Drying (g),Step6 Fermentation Sensor Data Phase3 Agitation Speed Val03 No1 (rounds per minutes),Step2 RawMat Internal Compound15 Number Openings (#),Step3 Media Age at Inoculation (days),Step3 RawMat Internal Compound15 Number Openings (#),Step4 RawMat Internal Compound01 QC (id),Step3 Media Compound17 Age at Inoculation (days),Step4-6 RawMat Internal Compound18 Age (days),Step6 RawMat Internal Compound04 Vendor (name),...,Step3 RawMat Vendor Compound05 Manufacturing Date (numeric date),Step1 Media Compound17 Weight (ml),Step5 Fermentation Sensor Data Phase1 Duration (minutes),Step6 Fermentation Sensor Data Phase3 Agitation Speed Sum No2 (rounds per minutes),Step6 Fermentation Sensor Data Phase3 Oxygen Val04 (%),Step6 Fermentation Sensor Data Phase3 Temperature Val03 (°C),Step6 Media Compound20 Age at Inoculation (days),Step4 RawMat Vendor Compound12 Manufacturing Date (numeric date),Step5 Fermentation Wait Duration (minutes),Step4-6 RawMat Internal Compound04 Age (days)
0,NaN,NaN,NaN,13.0,21.0,13.0,FP,36.0,NaN,COMPOUND PROVIDER INC,...,80953.0,174.907237,NaN,NaN,NaN,NaN,3.0,80963.0,767.0,NaN
1,NaN,NaN,NaN,13.0,21.0,13.0,FP,36.0,NaN,COMPOUND PROVIDER INC,...,80953.0,174.907237,NaN,NaN,NaN,NaN,3.0,80963.0,767.0,NaN
2,NaN,NaN,NaN,13.0,21.0,13.0,FP,36.0,NaN,COMPOUND PROVIDER INC,...,80953.0,174.907237,NaN,NaN,NaN,NaN,3.0,80963.0,767.0,NaN
3,790.284723,NaN,417.871938,13.0,25.0,13.0,FP,39.0,NaN,COMPOUND PROVIDER INC,...,80953.0,174.907237,NaN,97499.901386,31.394059,68.835293,4.0,80963.0,727.0,NaN
4,783.702715,NaN,413.294366,13.0,25.0,13.0,FP,39.0,NaN,COMPOUND PROVIDER INC,...,80953.0,174.907237,NaN,97190.471353,28.432164,68.764230,4.0,80963.0,727.0,NaN


In [8]:
TARGET = "Yield"
# define a Sample based on the test batch_file
sample = Sample(observations=batch_df, target_name=TARGET)
sample

In [9]:
# define the circular cross validator with 10 folds
circular_cv = CircularCrossValidator(test_ratio=0.2, num_splits=40)

circular_cv

CircularCrossValidator(num_splits=None, test_ratio=None)

In [10]:
if not CACHING:
    boruta_selector = PipelineDF(steps=[
        (
            'preprocess', 
            make_simple_transformer(
                impute_median_columns=sample.features_by_type(Sample.DTYPE_NUMERICAL).columns,
                one_hot_encode_columns=sample.features_by_type(Sample.DTYPE_OBJECT).columns,
            )
        ),
        (
            'boruta', 
            BorutaDF(
                estimator=RandomForestRegressor(n_jobs=4),
                max_iter=10,
                n_estimators='auto', 
                verbose=2, 
                random_state=42
            )
        )
    ])
    
    feature_selection = boruta_selector.fit_transform(
            sample.features, 
            sample.target
        )
    
    selected_features = boruta_selector.columns_original

    sample_post_boruta = sample.select_features(selected_features)
    
    selected_features.to_frame()

Iteration: 	1 / 10
Confirmed: 	0
Tentative: 	170
Rejected: 	0
Iteration: 	2 / 10
Confirmed: 	0
Tentative: 	170
Rejected: 	0
Iteration: 	3 / 10
Confirmed: 	0
Tentative: 	170
Rejected: 	0
Iteration: 	4 / 10
Confirmed: 	0
Tentative: 	170
Rejected: 	0
Iteration: 	5 / 10
Confirmed: 	0
Tentative: 	170
Rejected: 	0
Iteration: 	6 / 10
Confirmed: 	0
Tentative: 	170
Rejected: 	0
Iteration: 	7 / 10
Confirmed: 	0
Tentative: 	170
Rejected: 	0
Iteration: 	8 / 10
Confirmed: 	21
Tentative: 	37
Rejected: 	112


C:\Users\martin florent\AppData\Local\Continuum\anaconda3\lib\site-packages\boruta\boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 10
Confirmed: 	21
Tentative: 	37
Rejected: 	112


BorutaPy finished running.

Iteration: 	10 / 10
Confirmed: 	21
Tentative: 	14
Rejected: 	112


In [11]:
# define a transformer step based on the sample
if not CACHING:
    preprocessor = make_simple_transformer(
            impute_median_columns=sample_post_boruta.features_by_type(Sample.DTYPE_NUMERICAL).columns,
            one_hot_encode_columns=sample_post_boruta.features_by_type(Sample.DTYPE_OBJECT).columns,
    )
    preprocessor

In [12]:
if not CACHING:
    # define a Model with a preprocessing pipeline
    lgbm = ModelGrid(
                model=Model(
                    preprocessing=preprocessor, estimator=LGBMRegressor()
                ),
                estimator_parameters={
                    "max_depth": [5],
                    "min_split_gain": [0.1],
                    "num_leaves": [50],
                    "random_state": [42],
                    "n_estimators": [500]
                },
    )
    lgbm

In [13]:
if not CACHING:
    # define a Model with a preprocessing pipeline
    rf = ModelGrid(
                model=Model(
                    preprocessing=preprocessor, estimator=RandomForestRegressor(
                    verbose=0,
                    n_jobs=None
                    )
                ),
                estimator_parameters={
                    "max_depth": [5],
                    "random_state": [42],
                    "n_estimators": [500],
                },
    )
    rf

In [14]:
if not CACHING:
    # define a ModelRanker
    model_ranker: ModelRanker = ModelRanker(
            grids=[rf], cv=circular_cv, scoring="r2"
        )

    # run the ModelRanker to retrieve a ranking
    model_ranking = model_ranker.run(sample=sample_post_boruta)
    # noinspection PyStatementEffect
else:
    model_ranking = None
    
model_ranking

C:\Users\martin florent\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


[ModelEvaluation(model=<yieldengine.model.Model object at 0x000002912857A588>, parameters={'estimator__max_depth': 5, 'estimator__n_estimators': 500, 'estimator__random_state': 42}, scoring={'test_score': <yieldengine.model.selection.ModelScoring object at 0x0000029129A3FD30>}, ranking_score=-0.7491108296389388)]

In [15]:
if not CACHING:
    # retrieve the best model
    best_model = model_ranking[0]
    # noinspection PyStatementEffect
    best_model
else:
    best_model = None

In [16]:
best_model

ModelEvaluation(model=<yieldengine.model.Model object at 0x000002912857A588>, parameters={'estimator__max_depth': 5, 'estimator__n_estimators': 500, 'estimator__random_state': 42}, scoring={'test_score': <yieldengine.model.selection.ModelScoring object at 0x0000029129A3FD30>}, ranking_score=-0.7491108296389388)

In [17]:
if not CACHING:
    # define a ModelFitCV
    mp = ModelFitCV(
        model=best_model.model,
        cv=circular_cv,
        sample=sample
    )

    with open(MP_PKL, 'wb') as f:
        pickle.dump(mp, f)
else:
    with open(MP_PKL, 'rb') as f:
        mp = pickle.load(f)

mp

# Test Predictor.predictions_for_all_samples

In [18]:
best_model.model.estimator

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=5,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [19]:
import time

In [20]:
t = time.time()
predictor = ModelFitCV(model=best_model.model, cv=circular_cv, sample=sample, n_jobs=-1)
predictions = predictor.predictions_for_all_samples()
print(f"time: {time.time()-t} sec")

INFO:yieldengine.model.prediction:Finished to fit ModelFitCV.


time: 59.218138456344604 sec


In [21]:
t = time.time()
predictor = ModelFitCV(model=best_model.model, cv=circular_cv, sample=sample, n_jobs=1)
predictions = predictor.predictions_for_all_samples()
print(f"time:{time.time()-t} sec")

INFO:yieldengine.model.prediction:Finished to fit ModelFitCV.


time:108.96241068840027 sec


In [22]:
t = time.time()
predictor = ModelFitCV(model=best_model.model, cv=circular_cv, sample=sample, n_jobs=-2)
predictions = predictor.predictions_for_all_samples()
print(f"time: {time.time()-t} sec")

INFO:yieldengine.model.prediction:Finished to fit ModelFitCV.


time: 43.623528242111206 sec


In [23]:
t = time.time()
predictor = ModelFitCV(model=best_model.model, cv=circular_cv, sample=sample, n_jobs=-3)
predictions = predictor.predictions_for_all_samples()
print(f"time: {time.time()-t} sec")

INFO:yieldengine.model.prediction:Finished to fit ModelFitCV.


time: 36.52750301361084 sec
